[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/abigailhaddad/fedscope_new/blob/main/demo.ipynb)

# OPM Federal Workforce Data Explorer

This notebook loads federal workforce data from HuggingFace and lets you explore trends over time:
- **Accessions** - New federal hires
- **Separations** - Federal employee departures  
- **Employment** - Point-in-time workforce snapshots

**No authentication required** - all datasets are public.

In [ ]:
# Install dependencies (for Colab)
!pip install -q duckdb pandas plotly

In [ ]:
import duckdb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

## 1. Define Dataset URLs

The data is stored as parquet files on HuggingFace. DuckDB can query them directly via HTTP.

In [ ]:
# Generate all dataset URLs
HF_USERNAME = "abigailhaddad"
BASE_URL = f"https://huggingface.co/datasets/{HF_USERNAME}"

def generate_months(start_year=2021, start_month=1, end_year=2025, end_month=11):
    """Generate list of YYYYMM strings."""
    months = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            if year == start_year and month < start_month:
                continue
            if year == end_year and month > end_month:
                break
            months.append(f"{year}{month:02d}")
    return months

months = generate_months()

# Build URLs for each dataset
accession_urls = [f"{BASE_URL}/opm-federal-accessions-{m}/resolve/main/data.parquet" for m in months]
separation_urls = [f"{BASE_URL}/opm-federal-separations-{m}/resolve/main/data.parquet" for m in months]

print(f"Generated {len(accession_urls)} accession URLs")
print(f"Generated {len(separation_urls)} separation URLs")
print(f"\nExample: {accession_urls[0]}")

## 2. Load All Data into DuckDB

Load all parquet files once into DuckDB tables. After this, all queries are instant.

In [ ]:
%%time
# Create a persistent DuckDB connection
db = duckdb.connect()

# Load all accessions into one table
acc_url_list = ", ".join([f"'{url}'" for url in accession_urls])
db.execute(f"""
    CREATE TABLE accessions AS 
    SELECT *, 
           personnel_action_effective_date_yyyymm as month
    FROM read_parquet([{acc_url_list}])
""")

# Load all separations into one table  
sep_url_list = ", ".join([f"'{url}'" for url in separation_urls])
db.execute(f"""
    CREATE TABLE separations AS 
    SELECT *,
           personnel_action_effective_date_yyyymm as month
    FROM read_parquet([{sep_url_list}])
""")

# Check counts
acc_count = db.execute("SELECT SUM(CAST(count AS INTEGER)) FROM accessions").fetchone()[0]
sep_count = db.execute("SELECT SUM(CAST(count AS INTEGER)) FROM separations").fetchone()[0]

print(f"Loaded {acc_count:,} accessions and {sep_count:,} separations")
print(f"All data now in memory - filtering will be instant!")

## 3. Plot Accessions vs Separations Over Time

In [ ]:
# Aggregate by month for the overview chart (filter to 2021+ for complete data)
acc_df = db.execute("""
    SELECT month, SUM(CAST(count AS INTEGER)) as count
    FROM accessions
    WHERE month >= '202101'
    GROUP BY month
    ORDER BY month
""").df()
acc_df['date'] = pd.to_datetime(acc_df['month'], format='%Y%m')

sep_df = db.execute("""
    SELECT month, SUM(CAST(count AS INTEGER)) as count
    FROM separations
    WHERE month >= '202101'
    GROUP BY month
    ORDER BY month
""").df()
sep_df['date'] = pd.to_datetime(sep_df['month'], format='%Y%m')

# Plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=acc_df['date'], y=acc_df['count'],
    mode='lines+markers', name='Accessions (New Hires)',
    line=dict(color='#2ecc71', width=2), marker=dict(size=6),
    hovertemplate='%{x|%b %Y}<br>%{y:,.0f} new hires<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=sep_df['date'], y=sep_df['count'],
    mode='lines+markers', name='Separations (Departures)',
    line=dict(color='#e74c3c', width=2), marker=dict(size=6),
    hovertemplate='%{x|%b %Y}<br>%{y:,.0f} departures<extra></extra>'
))

fig.update_layout(
    title='Federal Workforce: Monthly Accessions vs Separations (2021-2025)',
    xaxis_title='Month', yaxis_title='Count',
    hovermode='x unified', template='plotly_white',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    yaxis=dict(tickformat=','), height=500
)
fig.show()

In [ ]:
# Net change (accessions - separations)
merged = acc_df.merge(sep_df, on=['month', 'date'], suffixes=('_acc', '_sep'))
merged['net_change'] = merged['count_acc'] - merged['count_sep']
merged['color'] = merged['net_change'].apply(lambda x: '#2ecc71' if x > 0 else '#e74c3c')

fig = go.Figure()

fig.add_trace(go.Bar(
    x=merged['date'],
    y=merged['net_change'],
    marker_color=merged['color'],
    hovertemplate='%{x|%b %Y}<br>Net: %{y:+,.0f}<extra></extra>'
))

fig.add_hline(y=0, line_width=1, line_color='black')

fig.update_layout(
    title='Federal Workforce: Monthly Net Change (Accessions - Separations)',
    xaxis_title='Month',
    yaxis_title='Net Change',
    template='plotly_white',
    yaxis=dict(tickformat=','),
    height=450,
    showlegend=False
)

fig.show()

print(f"\nTotal accessions: {merged['count_acc'].sum():,}")
print(f"Total separations: {merged['count_sep'].sum():,}")
print(f"Net change: {merged['net_change'].sum():+,}")

## 4. Explore the Data Structure

Let's look at what columns are available in the data.

In [ ]:
# Inspect columns
print("Accessions columns:")
print(db.execute("DESCRIBE accessions").df()['column_name'].tolist())

print("\nSeparations columns:")
print(db.execute("DESCRIBE separations").df()['column_name'].tolist())

In [ ]:
# Show sample data
print("Sample accession records:")
db.execute("SELECT * FROM accessions LIMIT 5").df()

## 5. Data Quality Analysis

Some fields have missing, redacted, or placeholder values that can affect analysis.

In [ ]:
# Analyze data quality across accession data (2021+ only for complete data)
print("=== DATA QUALITY ISSUES (accessions 2021-2025) ===\n")

total_people = db.execute("SELECT SUM(CAST(count AS INTEGER)) FROM accessions WHERE month >= '202101'").fetchone()[0]

# Only truly problematic values - REDACTED and NO DATA REPORTED
# Note: "ALL OTHER OCCUPATIONS" and "ALL OTHER POSITIONS" are valid categories, not data quality issues
checks = {
    'duty_station_state': ['REDACTED', 'NO DATA REPORTED'],
    'education_level': ['NO DATA REPORTED'],
}

for col, bad_values in checks.items():
    conditions = " OR ".join([f"{col} = '{v}'" for v in bad_values])
    
    result = db.execute(f"""
        SELECT {col} as value, SUM(CAST(count AS INTEGER)) as people
        FROM accessions
        WHERE month >= '202101' AND ({conditions})
        GROUP BY {col}
        ORDER BY people DESC
    """).df()
    
    if len(result) > 0:
        total_bad = result['people'].sum()
        pct = (total_bad / total_people) * 100
        
        print(f"📍 {col}: {total_bad:,.0f} people ({pct:.1f}%) have missing/redacted values")
        for _, row in result.iterrows():
            print(f"   • '{row['value']}': {row['people']:,.0f}")
        print()

# Show monthly breakdown for key data quality issues
print("\n=== MONTHLY DATA QUALITY TRENDS ===\n")

for col, bad_values in checks.items():
    conditions = " OR ".join([f"{col} = '{v}'" for v in bad_values])
    
    monthly = db.execute(f"""
        WITH totals AS (
            SELECT month, SUM(CAST(count AS INTEGER)) as total
            FROM accessions
            WHERE month >= '202101'
            GROUP BY month
        ),
        bad AS (
            SELECT month, SUM(CAST(count AS INTEGER)) as bad_count
            FROM accessions
            WHERE month >= '202101' AND ({conditions})
            GROUP BY month
        )
        SELECT t.month, 
               COALESCE(b.bad_count, 0) as bad_count,
               t.total,
               ROUND(100.0 * COALESCE(b.bad_count, 0) / t.total, 1) as pct
        FROM totals t
        LEFT JOIN bad b ON t.month = b.month
        ORDER BY t.month
    """).df()
    
    avg_pct = monthly['pct'].mean()
    min_pct = monthly['pct'].min()
    max_pct = monthly['pct'].max()
    min_month = monthly.loc[monthly['pct'].idxmin(), 'month']
    max_month = monthly.loc[monthly['pct'].idxmax(), 'month']
    
    print(f"📊 {col} (missing/redacted %):")
    print(f"   Average: {avg_pct:.1f}%")
    print(f"   Min: {min_pct:.1f}% ({min_month})")
    print(f"   Max: {max_pct:.1f}% ({max_month})")
    print()

In [ ]:
# Visualize data quality trends over time (2021+ only)
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=['Duty Station State: % Missing/Redacted', 'Education Level: % Missing'])

for i, (col, bad_values) in enumerate(checks.items()):
    conditions = " OR ".join([f"{col} = '{v}'" for v in bad_values])
    
    monthly = db.execute(f"""
        WITH totals AS (
            SELECT month, SUM(CAST(count AS INTEGER)) as total
            FROM accessions
            WHERE month >= '202101'
            GROUP BY month
        ),
        bad AS (
            SELECT month, SUM(CAST(count AS INTEGER)) as bad_count
            FROM accessions
            WHERE month >= '202101' AND ({conditions})
            GROUP BY month
        )
        SELECT t.month, 
               ROUND(100.0 * COALESCE(b.bad_count, 0) / t.total, 1) as pct
        FROM totals t
        LEFT JOIN bad b ON t.month = b.month
        ORDER BY t.month
    """).df()
    
    monthly['date'] = pd.to_datetime(monthly['month'], format='%Y%m')
    
    fig.add_trace(
        go.Scatter(
            x=monthly['date'], y=monthly['pct'],
            mode='lines+markers',
            name=col,
            hovertemplate='%{x|%b %Y}<br>%{y:.1f}%<extra></extra>'
        ),
        row=1, col=i+1
    )

fig.update_layout(
    title='Data Quality Trends: % of Records with Missing/Redacted Values by Month (2021-2025)',
    template='plotly_white',
    height=400,
    showlegend=False
)
fig.update_yaxes(title_text='% Missing/Redacted', row=1, col=1)
fig.update_yaxes(title_text='% Missing', row=1, col=2)
fig.show()

## 6. Slice the Data Your Way

The data is loaded into DuckDB tables called `accessions` and `separations`. You can query them directly using SQL.

### Available Fields

| Field | Description | Example Values |
|-------|-------------|----------------|
| `agency` | Federal agency | DEPARTMENT OF DEFENSE, DEPARTMENT OF VETERANS AFFAIRS |
| `duty_station_state` | Work location state | CALIFORNIA, TEXAS, DISTRICT OF COLUMBIA |
| `occupational_group` | Job category | MEDICAL AND HEALTH, INFORMATION TECHNOLOGY, LEGAL |
| `age_bracket` | Age range | 25-29, 30-34, 55-59 |
| `education_level` | Highest education | BACHELOR'S DEGREE, MASTER'S DEGREE, HIGH SCHOOL |
| `stem_occupation` | STEM job indicator | STEM, ALL OTHER OCCUPATIONS |
| `supervisory_status` | Management level | SUPERVISOR, MANAGER, ALL OTHER POSITIONS |
| `month` | YYYYMM format | 202101, 202506 |
| `count` | Number of people | (aggregated records) |

### Query Pattern

```python
# Filter accessions by any field
df = db.execute("""
    SELECT month, SUM(CAST(count AS INTEGER)) as count
    FROM accessions
    WHERE month >= '202101'
      AND your_field = 'your_value'
    GROUP BY month
    ORDER BY month
""").df()
```

In [ ]:
# See what values are available for each field
print("=== TOP VALUES BY FIELD ===\n")

fields_to_show = ['agency', 'duty_station_state', 'occupational_group', 'education_level']

for field in fields_to_show:
    result = db.execute(f"""
        SELECT {field} as value, SUM(CAST(count AS INTEGER)) as n
        FROM accessions
        WHERE month >= '202101' AND {field} IS NOT NULL AND {field} != ''
        GROUP BY {field}
        ORDER BY n DESC
        LIMIT 10
    """).df()
    
    print(f"📊 {field}:")
    for _, row in result.iterrows():
        print(f"   {row['value']}: {row['n']:,}")
    print()

In [ ]:
# Example: Accessions by top 5 agencies
top_agencies = db.execute("""
    SELECT agency, SUM(CAST(count AS INTEGER)) as n
    FROM accessions
    WHERE month >= '202101'
    GROUP BY agency
    ORDER BY n DESC
    LIMIT 5
""").df()['agency'].tolist()

fig = go.Figure()

for agency in top_agencies:
    df = db.execute(f"""
        SELECT month, SUM(CAST(count AS INTEGER)) as count
        FROM accessions
        WHERE month >= '202101' AND agency = '{agency}'
        GROUP BY month
        ORDER BY month
    """).df()
    df['date'] = pd.to_datetime(df['month'], format='%Y%m')
    
    fig.add_trace(go.Scatter(
        x=df['date'], y=df['count'],
        mode='lines', name=agency[:30],
        hovertemplate='%{x|%b %Y}<br>%{y:,.0f}<extra></extra>'
    ))

fig.update_layout(
    title='Monthly Accessions by Top 5 Agencies',
    xaxis_title='Month', yaxis_title='New Hires',
    template='plotly_white', height=500,
    yaxis=dict(tickformat=','),
    legend=dict(orientation='h', yanchor='bottom', y=-0.3)
)
fig.show()

In [ ]:
# Example: Accessions by top 5 states
top_states = db.execute("""
    SELECT duty_station_state, SUM(CAST(count AS INTEGER)) as n
    FROM accessions
    WHERE month >= '202101' 
      AND duty_station_state NOT IN ('REDACTED', 'NO DATA REPORTED')
    GROUP BY duty_station_state
    ORDER BY n DESC
    LIMIT 5
""").df()['duty_station_state'].tolist()

fig = go.Figure()

for state in top_states:
    df = db.execute(f"""
        SELECT month, SUM(CAST(count AS INTEGER)) as count
        FROM accessions
        WHERE month >= '202101' AND duty_station_state = '{state}'
        GROUP BY month
        ORDER BY month
    """).df()
    df['date'] = pd.to_datetime(df['month'], format='%Y%m')
    
    fig.add_trace(go.Scatter(
        x=df['date'], y=df['count'],
        mode='lines', name=state,
        hovertemplate='%{x|%b %Y}<br>%{y:,.0f}<extra></extra>'
    ))

fig.update_layout(
    title='Monthly Accessions by Top 5 States',
    xaxis_title='Month', yaxis_title='New Hires',
    template='plotly_white', height=500,
    yaxis=dict(tickformat=','),
    legend=dict(orientation='h', yanchor='bottom', y=-0.2)
)
fig.show()

In [ ]:
# Example: Compare accessions vs separations for a specific agency
agency_name = 'DEPARTMENT OF VETERANS AFFAIRS'

acc = db.execute(f"""
    SELECT month, SUM(CAST(count AS INTEGER)) as count
    FROM accessions
    WHERE month >= '202101' AND agency = '{agency_name}'
    GROUP BY month ORDER BY month
""").df()
acc['date'] = pd.to_datetime(acc['month'], format='%Y%m')

sep = db.execute(f"""
    SELECT month, SUM(CAST(count AS INTEGER)) as count
    FROM separations
    WHERE month >= '202101' AND agency = '{agency_name}'
    GROUP BY month ORDER BY month
""").df()
sep['date'] = pd.to_datetime(sep['month'], format='%Y%m')

fig = go.Figure()
fig.add_trace(go.Scatter(x=acc['date'], y=acc['count'], mode='lines+markers', 
                         name='Accessions', line=dict(color='#2ecc71')))
fig.add_trace(go.Scatter(x=sep['date'], y=sep['count'], mode='lines+markers', 
                         name='Separations', line=dict(color='#e74c3c')))

fig.update_layout(
    title=f'Accessions vs Separations: {agency_name}',
    xaxis_title='Month', yaxis_title='Count',
    template='plotly_white', height=450,
    yaxis=dict(tickformat=','),
    hovermode='x unified'
)
fig.show()

# Net change
total_acc = acc['count'].sum()
total_sep = sep['count'].sum()
print(f"\n{agency_name}")
print(f"Total accessions: {total_acc:,}")
print(f"Total separations: {total_sep:,}")
print(f"Net change: {total_acc - total_sep:+,}")

## 7. Employment Data (Point-in-Time Snapshots)

The employment data shows the full federal workforce at a point in time (not just hires/departures). These files are larger (~30MB each), so we'll only load recent months.

In [ ]:
%%time
# Load only 2025 employment data (smaller subset)
employment_months = ['202501', '202502', '202503', '202504', '202505', '202506', '202507', '202508', '202509', '202510', '202511']
employment_urls = [f"{BASE_URL}/opm-federal-employment-{m}/resolve/main/data.parquet" for m in employment_months]

# Load into DuckDB
emp_url_list = ", ".join([f"'{url}'" for url in employment_urls])
db.execute(f"""
    CREATE OR REPLACE TABLE employment AS 
    SELECT *, snapshot_yyyymm as month
    FROM read_parquet([{emp_url_list}])
""")

emp_count = db.execute("SELECT SUM(CAST(count AS INTEGER)) FROM employment").fetchone()[0]
print(f"Loaded {emp_count:,} employee records across {len(employment_months)} months")

In [ ]:
# What columns are in employment data?
print("Employment columns:")
print(db.execute("DESCRIBE employment").df()['column_name'].tolist())

In [ ]:
# Example: IT Specialists (2210 series) at DHS over time
df = db.execute("""
    SELECT month, SUM(CAST(count AS INTEGER)) as employees
    FROM employment
    WHERE agency LIKE '%HOMELAND SECURITY%'
      AND occupation LIKE '2210%'
    GROUP BY month
    ORDER BY month
""").df()
df['date'] = pd.to_datetime(df['month'], format='%Y%m')

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['date'], y=df['employees'],
    mode='lines+markers',
    line=dict(color='#3498db', width=2),
    hovertemplate='%{x|%b %Y}<br>%{y:,.0f} employees<extra></extra>'
))
fig.update_layout(
    title='IT Specialists (2210 Series) at DHS - 2025',
    xaxis_title='Month', yaxis_title='Employees',
    template='plotly_white', height=400,
    yaxis=dict(tickformat=',')
)
fig.show()

print(f"Latest count: {df['employees'].iloc[-1]:,} IT specialists at DHS")

In [ ]:
# Example: Total federal workforce by top agencies - 2025
top_agencies = db.execute("""
    SELECT agency, SUM(CAST(count AS INTEGER)) as n
    FROM employment
    WHERE month = '202511'
    GROUP BY agency
    ORDER BY n DESC
    LIMIT 6
""").df()['agency'].tolist()

fig = go.Figure()
for agency in top_agencies:
    df = db.execute(f"""
        SELECT month, SUM(CAST(count AS INTEGER)) as employees
        FROM employment
        WHERE agency = '{agency}'
        GROUP BY month
        ORDER BY month
    """).df()
    df['date'] = pd.to_datetime(df['month'], format='%Y%m')
    
    fig.add_trace(go.Scatter(
        x=df['date'], y=df['employees'],
        mode='lines+markers', name=agency[:35],
        hovertemplate='%{x|%b %Y}<br>%{y:,.0f}<extra></extra>'
    ))

fig.update_layout(
    title='Total Workforce by Top Agencies - 2025',
    xaxis_title='Month', yaxis_title='Employees',
    template='plotly_white', height=500,
    yaxis=dict(tickformat=','),
    legend=dict(orientation='h', yanchor='bottom', y=-0.4)
)
fig.show()

## 8. Try Your Own Queries

All three tables are now available: `accessions`, `separations`, and `employment`.

**Common occupation codes:**
- `2210` - IT Specialist
- `0343` - Management Analyst  
- `0301` - Miscellaneous Administration
- `0905` - Attorney
- `1102` - Contract Specialist

**Example queries to try:**

```python
# Total workforce at a specific agency
db.execute("""
    SELECT month, SUM(CAST(count AS INTEGER)) as employees
    FROM employment
    WHERE agency LIKE '%DEFENSE%'
    GROUP BY month ORDER BY month
""").df()

# Attorneys hired in 2025
db.execute("""
    SELECT month, SUM(CAST(count AS INTEGER)) as hires
    FROM accessions
    WHERE occupation LIKE '0905%' AND month >= '202501'
    GROUP BY month ORDER BY month
""").df()

# Compare two states
db.execute("""
    SELECT duty_station_state, SUM(CAST(count AS INTEGER)) as employees
    FROM employment
    WHERE month = '202511' 
      AND duty_station_state IN ('CALIFORNIA', 'TEXAS')
    GROUP BY duty_station_state
""").df()
```